In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def distance(point1, point2,distances_matrix):
    return distances_matrix[point1-1][point2-1]

def check_convergence(paths):
    p=paths.count(paths[0])/len(paths)
    for path in paths:
        if paths.count(path)/len(paths) > p:
            p=paths.count(path)/len(paths)
    return p

In [3]:
def ant_colony_optimization(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,distances_matrix):
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = None
    best_path_length = np.inf
    
    for iteration in range(n_iterations):
        paths = []
        path_lengths = []
        
        for ant in range(n_ants):
            visited = [False]*n_points
            current_point = np.random.randint(n_points)
            visited[current_point] = True
            path = [current_point]
            path_length = 0
            
            while False in visited:
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))
                
                for i, unvisited_point in enumerate(unvisited):
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point],distances_matrix)**beta
                
                probabilities /= np.sum(probabilities)
                
                next_point = np.random.choice(unvisited, p=probabilities)
                path.append(next_point)
                path_length += distance(points[current_point], points[next_point],distances_matrix)
                visited[next_point] = True
                current_point = next_point
            
            path_length=path_length+distance(points[0], points[-1],distances_matrix)
            
            paths.append(path)
            path_lengths.append(path_length)
            
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length
                
            
        
        pheromone *= evaporation_rate
        
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
        
        convergence=check_convergence(paths)

        if convergence < 0.7:
            print("iteration number "+ str(iteration+1)+" convergence "+str(convergence)+" best solution "+str(best_path_length)+" best path "+str(best_path))
        else:
            print("ACO convergence in iteration number"+str(iteration+1)+" best solution "+str(best_path_length)+" best path "+str(best_path))
            break
    


#best solution 291
matrix = np.loadtxt("15city.txt")


points = np.array([i+1 for i in range(len(matrix))]) 


ant_colony_optimization(points, n_ants=20, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5, Q=1 ,distances_matrix=matrix)

iteration number 1 convergence 0.05 best solution 398.0 best path [3, 5, 13, 11, 2, 6, 8, 14, 1, 12, 0, 10, 7, 4, 9]
iteration number 2 convergence 0.05 best solution 324.0 best path [0, 10, 3, 5, 7, 9, 1, 12, 14, 8, 4, 6, 2, 11, 13]
iteration number 3 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 4 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 5 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 6 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 7 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 8 convergence 0.05 best solution 316.0 best path [14, 8, 4, 6, 2, 11, 13, 9, 7, 1, 12, 0, 10, 3, 5]
iteration number 9 convergence 0.05 best solution 316.0 best pat

In [5]:
def ant_colony_optimization(points, n_ants_list, n_iterations, alpha, beta, evaporation_rate, Q,distances_matrix):
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path_lengths = []
    
    for n_ants in n_ants_list:
        best_path = None
        best_path_length = np.inf
        elitist_path = None
        elitist_path_length = np.inf
        
        for iteration in range(n_iterations):
            paths = []
            path_lengths = []
            convergence=0

            for ant in range(n_ants):
                visited = [False]*n_points
                current_point = np.random.randint(n_points)
                visited[current_point] = True
                path = [current_point]
                path_length = 0

                while False in visited:
                    unvisited = np.where(np.logical_not(visited))[0]
                    probabilities = np.zeros(len(unvisited))

                    for i, unvisited_point in enumerate(unvisited):
                        probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point],distances_matrix)**beta

                    probabilities /= np.sum(probabilities)

                    next_point = np.random.choice(unvisited, p=probabilities)
                    path.append(next_point)
                    path_length += distance(points[current_point], points[next_point],distances_matrix)
                    visited[next_point] = True
                    current_point = next_point

                path_length=path_length+distance(points[0], points[-1],distances_matrix)

                paths.append(path)
                path_lengths.append(path_length)

                if path_length < best_path_length:
                    best_path = path
                    best_path_length = path_length

                if path_length < elitist_path_length:
                    elitist_path = path
                    elitist_path_length = path_length

            pheromone *= evaporation_rate

            for path, path_length in zip(paths, path_lengths):
                for i in range(n_points-1):
                    pheromone[path[i], path[i+1]] += Q/path_length
                pheromone[path[-1], path[0]] += Q/path_length
                
            for i in range(n_points-1):
                pheromone[elitist_path[i], elitist_path[i+1]] += Q/elitist_path_length
            pheromone[elitist_path[-1], elitist_path[0]] += Q/elitist_path_length

            convergence=check_convergence(paths)
            if convergence > 0.7:
                break
            
        best_path_lengths.append((convergence,best_path_length,best_path))
        
    return best_path_lengths

matrix = np.loadtxt("15city.txt")
points = np.array([i+1 for i in range(len(matrix))])

n_ants_list = [10, 40, 80]

best_solution =ant_colony_optimization(points, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5, Q=1,distances_matrix=matrix,n_ants_list=n_ants_list)

for i in range(len(n_ants_list)):
    print("n_ants "+str(n_ants_list[i])+" best solution "+str(best_solution[i]))

n_ants 10 best solution (0.2, 312.0, [10, 3, 5, 7, 9, 13, 11, 2, 6, 4, 14, 8, 1, 12, 0])
n_ants 40 best solution (0.175, 312.0, [10, 3, 5, 7, 9, 13, 11, 2, 6, 4, 14, 8, 1, 12, 0])
n_ants 80 best solution (0.125, 312.0, [10, 3, 5, 7, 9, 13, 11, 2, 6, 4, 14, 8, 1, 12, 0])
